# huntabyte/bits-ui

> 

In [1]:
#| default_exp huntabyte-bitsui

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()

In [3]:
#| export

from lovely_docs.settings import settings, GitSource

from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import shutil
from pathlib import Path
import logging
import subprocess
import os

from lovely_docs.git import clone_repo

In [4]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
#| export

build = True

source = GitSource(
    name="huntabyte/bitsui",
    doc_dir="docs/static/docs",
    repo="git@github.com:xl0/bits-ui.git",
    commit="better-llms-txt",
    ecosystems=["svelte"])

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

In [6]:
if build:
    with open("huntabyte-bitsui.log", "w") as log_file:
        # Clean the repo
        subprocess.run(["git", "clean", "-dfx"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Install dependencies in root
        subprocess.run(["pnpm", "install"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Build the package
        subprocess.run(["pnpm", "build"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Install dependencies in docs
        subprocess.run(["pnpm", "install"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

        # Build the docs
        subprocess.run(["pnpm", "build"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

In [7]:
#| export

# Recursively copy all .txt files in docs/static to corresponding .md files
static_dir = clone_dir / "docs" / "static" / "docs"
for txt_file in static_dir.rglob("*.txt"):
    md_file = txt_file.with_suffix(".md")
    md_file.parent.mkdir(parents=True, exist_ok=True)
    # logger.debug(f"{txt_file} -> {md_file}")
    shutil.copy(txt_file, md_file)

os.remove(clone_dir / "docs" / "static" / "docs"/ "llms.md")

In [8]:
#| export

tree = build_markdown_doc_tree(clone_dir/source.doc_dir)

In [9]:
# Path("bits.json").write_text(tree.model_dump_json(indent=2))

In [10]:
# tree.subdirs = tree.subdirs[:3]
# tree.subdirs[1].subdirs = tree.subdirs[1].subdirs[:4]

#### Split the single llms.txt file into pages and directories based on heading levels

In [11]:
# Path("bits-compact.json").write_text(compacted_tree.model_dump_json(indent=2))

In [12]:
#| export

processed_tree = await process_tree_depth_first(settings, tree, source.name)

+279.699s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/accordion/llms.md
+0.307s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/alert-dialog/llms.md
+0.015s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/aspect-ratio/llms.md
+0.015s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/avatar/llms.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/button/llms.md
+0.015s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/calendar/llms.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/checkbox/llms.md
+0.015s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/collapsible/llms.md
+0.015s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/combobox/llms.md
+0.015s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/command/llms.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_p

#### Calculate cost

In [13]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 497,199
  Output tokens: 66,817
  Total tokens: 564,016
Cost:
  Total: $0.83
  Input: $0.50
  Output: $0.33


#### save the results

In [14]:
#| export

save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)